# Введение в рекомендательные системы

## Коллаборативная фильтрация

В этом задании мы закончим имплементацию коллаборативной фильтрации.

Для этого - выполним действия, необходимые для создания матрицы рейтингов

In [289]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from tqdm import tqdm_notebook
import math

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

In [290]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [291]:
filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

# Перекодируем ID фильмов и пользователей

In [292]:
from sklearn.preprocessing import LabelEncoder

In [293]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [294]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [295]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [296]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)
df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

In [297]:
df_rates.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,0,2,1.0,29,10,2006,23,17,16
1,0,31,4.5,29,10,2006,23,23,44
2,0,105,4.0,29,10,2006,23,30,8
3,0,151,2.0,29,10,2006,23,16,52
4,0,154,4.0,29,10,2006,23,29,30


## Матрица рейтингов

In [298]:
from scipy.sparse import coo_matrix, csr_matrix

In [299]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [300]:
R.shape

(2113, 10109)

## Похожесть между пользователями

В дальнейшем нам будет удобнее работать с форматом `Compressed Sparse Row matrix`. К счастью переформатировать полученную нами матрицу можно одной командой:

In [301]:
R = R.tocsr()

Теперь, например, рейтинги для первого пользователя можно достать так:

In [302]:
user_1 = R[0]
user_1

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 55 stored elements in Compressed Sparse Row format>

Так как вы возможно не работали с разреженным форматом матриц, устроим небольшой ликбез.

Первым делом, надо понадобится вектор для другого пользователя:

In [303]:
user_2 = R[1]
user_2.shape

(1, 10109)

Мы можем сравнивать элементы с 0

In [304]:
user_1_rated = (user_1 != 0)

Можем их "индексировать"

In [305]:
user_1[user_1_rated]

matrix([[1. , 4.5, 4. , 2. , 4. , 4.5, 3.5, 5. , 3.5, 2. , 4. , 3. , 4.5,
         0.5, 4.5, 4. , 3.5, 4.5, 4. , 2.5, 4. , 4. , 4. , 4.5, 2.5, 2. ,
         1.5, 4. , 4. , 4.5, 3. , 3. , 4.5, 3.5, 4.5, 1.5, 3. , 3. , 3.5,
         3.5, 3. , 2.5, 3.5, 4. , 0.5, 4. , 3.5, 4.5, 3.5, 4.5, 5. , 3.5,
         3.5, 3.5, 4.5]])

Можем считать количество ненулевых элементов

In [306]:
user_1.nnz

55

In [307]:
user_2 = R[2]

In [308]:
user_2_rated = (user_2 != 0)

Можем умножать 2 разреженных вектора поэлементно:

In [309]:
both_rated = (user_1_rated).multiply(user_2_rated)

In [310]:
both_rated.shape

(1, 10109)

In [311]:
user_1[both_rated]

<1x0 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [312]:
user_2[both_rated]

<1x0 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [313]:
np.multiply(user_1[both_rated], user_2[both_rated]).sum()

ValueError: dimension mismatch

In [ ]:
user_1[both_rated].dot(user_1[both_rated].T)[0, 0]

и скалярно

In [ ]:
user_1.dot(user_2.T)[0, 0]

И превращать разреженную матрицу (вектор) в плотную

In [ ]:
user_1_dense = user_1.toarray()
user_1_dense

Этого ликбеза вам будет должно быть достаточно, чтобы реализовать функцию расчета похожести между парой пользователей $u$ и $v$:

$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$

Давайте будем считать, что если количество фильмов которые пользователь $u$ и $v$ посмотрели вместе $<= 2$, то их косинусная мера равна 0.0


In [314]:
def cosine_similarity_pair_users(u, v, R):
    user_1 = R[u]
    user_2 = R[v]
    
    user_1_rated = (user_1 != 0)
    user_2_rated = (user_2 != 0)
    
    both_rated = (user_1_rated).multiply(user_2_rated)
    
    user_1_both_rated = user_1[both_rated]    
    if user_1_both_rated.shape[1] <= 2:
        return 0.0    
    user_2_both_rated = user_2[both_rated]
    
    a = user_1.dot(user_2.T)[0, 0]
    b = user_1_both_rated.dot(user_1_both_rated.T)[0, 0]
    c = user_2_both_rated.dot(user_2_both_rated.T)[0, 0]
    s = a / (math.sqrt(b) * math.sqrt(c))
    
    return s

In [316]:
cosine_similarity_pair_users(1,2, R)

0.0

### _Решение_

## Функция прогнозирования рейтинга

Реализуйте функцию, которая принимает на входе
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей (несмотря на то, что каждый пользователь - ближайший сосед самому себе, в расчетах он использоваться не должен)

и возвращает вектор с предсказанными рейтингами по всем товарам для этого пользователя

Для того, чтобы считать прогноз по рейтингу мы воспользуемся упрощенной формулой из лекции:

$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv}R_{vi}}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$


### _Решение_

In [ ]:
def get_rating(u, R, k):
    all_u = []
    for i in range(0, R.shape[0]):
        if i == u: 
            all_u.append(0.0)
        else:
            sim = cosine_similarity_pair_users(u, i, R) 
            all_u.append(sim)
        
    N_u = np.argpartition(all_u, -k)[-k:]    
    print(N_u)
    print(N_u[1])
    print(all_u[2])
    print(all_u[0:10])

In [ ]:
get_rating(1, R, 5)

В качестве ответа к этому заданию верните 5 идентификаторов фильмов с наивысшим предсказанным рейтингом для пользователя с id 19 (20-я строчка в матрице рейтингов).
* Для усреднения используйте 30 ближайших соседей
* Среди этих 5-и фильмов не должно быть ранее просмотренных фильмов

### _Решение_

## Генерация ответа

In [ ]:
', '.join(str(i) for i in top5)